# **Introduction**

In this project I will apply supervised machine learning techniques on the US census data to help a fictitious charity organization(CharityML) identify people most likely to donate to their cause.
Firstly I will perform some preprocessing transformations in order to  to manipulate the data into a workable format. Next, I will evaluate few algorithms of on the data, and consider which is best suited for the solution. Afterwards, I will optimize the selected model and present it as my solution to CharityML.

The success of the model will be determined based on the model's AUC or area under the curve associated with ROC curves.

In [ ]:
# importing the necessary libraries

import time
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from imblearn.over_sampling import SMOTE

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

import xgboost as xgb

import optuna

from sklearn.metrics import accuracy_score, precision_score,recall_score,confusion_matrix, roc_auc_score

In [ ]:
START = time.time()

In [ ]:
df=pd.read_csv('../input/udacity-mlcharity-competition/census.csv')
df.head()

## Training Data Exploration

In [ ]:
df.info()

The training data does not contains missing values. It consist of a mix of categorical and numerical features.

In [ ]:
# checking the label column value distribution

df.income.value_counts()

The target column has a significant imbalance in favor of the <=50K class, which will compromise the model training especially for the lower rapresented class.
In order to overcome this problem I can try the following:
* Use learners capable of dealing with inbalances (i.e. XGBoost)
* Use oversampling techniques such us SMOTE to create an omogeneous trainig dataset

In [ ]:
# Replacing the income col values for labels.

df.income.replace({'<=50K':0,'>50K':1}, inplace=True)

features = df.drop(columns=['income'])
label = df['income']

### Numerical features

In [ ]:
numerical = ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

for col in numerical:
    print('\n')
    plt.hist(df[col])
    plt.title(col)
    plt.show()
    print(col + ' skewness: ',df[col].skew())

As shown by the plots and by the skewness value, *capital-gain* and *capital-loss* are highly skewed.
In this cases, using a logarithmic transformation significantly reduces the range of values caused by outliers,so that the very large and very small values do not negatively affect the performance of a learning algorithm. However, I must transoform the values by a small amount above 0 due to the fact that the logarithm of 0 is undefined.

In [ ]:
# Log-transformation of the skewed features

skewed = ['capital-gain', 'capital-loss']

features_log = features
features_log[skewed] = features[skewed].apply(lambda x: np.log(x+1))

In [ ]:
# Normalization of the numerical features
scl = MinMaxScaler()
features_scaled = features_log
features_scaled[numerical] = scl.fit_transform(features_log[numerical])

### Categorical features

In [ ]:
#One-Hot-Encoding of the categorical features

features_final = pd.get_dummies(features_scaled)

In [ ]:
features_final

# Kaggle Test

Now I take a look at the Kaggle test set

In [ ]:
kaggle = pd.read_csv('../input/udacity-mlcharity-competition/test_census.csv')

In [ ]:
kaggle.info()

Upon inspection of the kaggle test data, I found an extra field named *Unamed: 0* and also that every other features contains missing values.
I will now proceed in dropping the extra column and filling the missing values with the training dataset most frequent value for that same column.

Afterwards, I will apply the same trasformations for both numerical and categorical features accordingly utilized in the training data.

In [ ]:
# dropping the extra column

kaggle.drop(columns=['Unnamed: 0'], inplace=True)
kaggle

In [ ]:
for col in kaggle.columns:
    kaggle[col].fillna(features[col].mode()[0], inplace=True)

In [ ]:
'''for col in kaggle.columns:

    kaggle[col].fillna(kaggle[col].mode()[0], inplace=True)'''

In [ ]:
# Applying the same transformations used in the training data

skewed = ['capital-gain', 'capital-loss']
numerical = ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

# Log-transformation of the skewed featurez
featurez_log = kaggle
featurez_log[skewed] = featurez_log[skewed].apply(lambda x: np.log(x+1))

# Applying the scaler to the featurez
featurez_scaled = featurez_log
featurez_scaled[numerical] = scl.fit_transform(featurez_log[numerical])

# OneHotEncoding of the categorical features
featurez_final = pd.get_dummies(featurez_scaled)

# Modeling

In [ ]:
# Split the 'features' and 'income' data into training and testing sets
X_train, X_val, y_train, y_val = train_test_split(features_final, 
                                                    label, 
                                                    test_size = 0.2)

In [ ]:
# calculating the pos_weight value for xgboost

pos_weight = round((df.income.value_counts()[0])/(df.income.value_counts()[1]))
pos_weight

In [ ]:
# Applying SMOTE to the training set in order to resolve the target variable values inbalance

sm = SMOTE()
X_train, y_train = sm.fit_resample(X_train,y_train)

y_train.value_counts()

It is now time to run few learners on the training data in order to compare their performances. 
I will test the performance of four learners: AdaBoost, RandomForest, XGBoost and SVM.
I will then create a voting model with the best three in order to create the predictions for the submission.

In [ ]:
# Initializing the learners

ada = AdaBoostClassifier(base_estimator=DecisionTreeClassifier())
rf = RandomForestClassifier()
xgb_boost = xgb.XGBClassifier(eval_metric='logloss', tree_method='gpu_hist')
svm = SVC(probability=True)

clf_list = [ada, rf, xgb_boost,svm]
clf_names = ['AdaBoost', 'Random Forest', 'XGBoost','SVM']

In [ ]:
start_time = time.time()

results = {'Algorithm' : 'xxx',
      'Accuracy':'xxx',
      'Precision':'xxx',
      'Recall':'xxx'}

results = pd.DataFrame(results, index=[0])

for n, clf in enumerate(clf_list):
    
    clf.fit(X_train, y_train)
    y_preds = clf.predict(X_val)
    y_pred_probs = clf.predict_proba(X_val)[:,1]
    
    rw = {'Algorithm' : clf_names[n],
          'Accuracy':accuracy_score(y_val, y_preds),
          'Precision':precision_score(y_val, y_preds),
          'Recall':recall_score(y_val, y_preds), 
          'ROC':roc_auc_score(y_val,y_pred_probs)}
    
    rw = pd.DataFrame(rw, index=[0])
    results = pd.concat([results,rw])
    
    print('\n\n')
    print(clf_names[n] + ' Confusion Matrix')
    print(confusion_matrix(y_val, y_preds))
    

end_time = time.time()
print('\n',time.strftime("%Hh%Mm%Ss", time.gmtime((end_time-start_time))))

In [ ]:
results = results.iloc[1:]
results

In [ ]:
# Viszualizing  model performaces

fig,(ax1,ax2,ax3,ax4) = plt.subplots(1,4,figsize=(20,8))

ax1.bar(x=results['Algorithm'], height=results['Precision'])
ax1.title.set_text('Precision')
ax1.tick_params(axis='x', rotation=45)
ax2.bar(x=results['Algorithm'], height=results['Recall'])
ax2.title.set_text('Recall')
ax2.tick_params(axis='x', rotation=45)
ax3.bar(x=results['Algorithm'], height=results['Accuracy'])
ax3.title.set_text('Accuracy')
ax3.tick_params(axis='x', rotation=45)
ax4.bar(x=results['Algorithm'], height=results['ROC'])
ax4.title.set_text('ROC AUC score')
ax4.tick_params(axis='x', rotation=45);

* **XGBoost:** is the best perfoming learner overall
* **AdaBoost:** has a better performance for the recall than the precision. Good also the ROC value
* **RandomForest:** is the second best performing learner overall. Similar scores to AdaBoost, just a tiny better
* **SVM:** the best recall score of the four learners, but the worse precision.Good value for the ROC, however the training takes significantly more time.

In order to carry out the voting between the best three performers, I create a function which averages the output of the three learners. Taking the average is important as I will use predict_proba() instead of predict(). The former is a more accurate way of predicting when the evaluation metric is the AUC or area under the curve associated with ROC curves.

In [ ]:
def voting_predictor(clf_1,clf_2, clf_3,cols_names,file_name):
    
    y1_pred = clf_1.predict_proba(featurez_final)[:,1].reshape(featurez_final.shape[0],1)
    y2_pred = clf_2.predict_proba(featurez_final)[:,1].reshape(featurez_final.shape[0],1)
    y3_pred = clf_3.predict_proba(featurez_final)[:,1].reshape(featurez_final.shape[0],1)
    
    y_pred = pd.DataFrame(np.concatenate([y1_pred,y2_pred,y3_pred], axis=1),columns=cols_names)
    y_pred = y_pred.mean(axis=1, numeric_only=True).reset_index()
    y_pred.columns = ['id','income']
    
    y_pred.to_csv(file_name, index=False)
  

    
    y1_pred_test = clf_1.predict_proba(X_val)[:,1].reshape(X_val.shape[0],1)
    y2_pred_test = clf_2.predict_proba(X_val)[:,1].reshape(X_val.shape[0],1)
    y3_pred_test = clf_3.predict_proba(X_val)[:,1].reshape(X_val.shape[0],1)
    
    y_pred_test = pd.DataFrame(np.concatenate([y1_pred_test,y2_pred_test,y3_pred_test], axis=1),columns=cols_names)
    y_pred_test = y_pred_test.mean(axis=1, numeric_only=True)
    
    print('\n',roc_auc_score(y_val,y_pred_test))
    
    return print('DONE!')

# XGBoost Optimization

I now carry out the optimization of the learners using Optuna and ROC as scoring metric.

**Optuna SearchCV**



In [ ]:
param_distributions = {
    'eta': optuna.distributions.UniformDistribution(0.01,0.1),
    'max_depth': optuna.distributions.IntUniformDistribution(4,10),
    'gamma': optuna.distributions.UniformDistribution(0,5),
    'n_estimators': optuna.distributions.IntUniformDistribution(150,1500),
    'booster': optuna.distributions.CategoricalDistribution(['gbtree', 'dart']),
    'pos_weight': optuna.distributions.UniformDistribution(0,pos_weight)
}

In [ ]:
start_time = time.time()


xgb_opt_search = optuna.integration.OptunaSearchCV(estimator=xgb_boost,
                                               param_distributions = param_distributions,
                                               cv=5,
                                               n_jobs=-1,
                                               n_trials = 10,
                                               scoring='roc_auc',
                                               verbose=10)

xgb_opt_search.fit(X_train, y_train)
y_preds = xgb_opt_search.predict_proba(X_val)[:,1]

print('\n','Best Score',xgb_opt_search.best_score_)
print('\n','Best Params',xgb_opt_search.best_estimator_.get_params())
print('\n',roc_auc_score(y_val,y_preds))

end_time = time.time()
print('\n',time.strftime("%Hh%Mm%Ss", time.gmtime((end_time-start_time))))

# AdaBoost optimization

**Optuna SearchCV**

In [ ]:
param_distributions = {
    'base_estimator__criterion': optuna.distributions.CategoricalDistribution(['gini','entropy']),
    'base_estimator__max_features':optuna.distributions.CategoricalDistribution(['auto','sqrt','log2']),
    'base_estimator__max_depth':optuna.distributions.IntUniformDistribution(4,15),
    'n_estimators':optuna.distributions.IntUniformDistribution(150,1500),
    'learning_rate':optuna.distributions.UniformDistribution(0.01,0.1)
}

In [ ]:
start_time = time.time()

ada_opt_search = optuna.integration.OptunaSearchCV(estimator=ada,
                                               param_distributions = param_distributions,
                                               cv=5,
                                               n_jobs=-1,
                                               n_trials = 10,
                                               scoring='roc_auc',
                                               verbose=10)

ada_opt_search.fit(X_train, y_train)
y_preds = ada_opt_search.predict_proba(X_val)[:,1]

print('\n','Best Score',ada_opt_search.best_score_)
print('\n','Best Params',ada_opt_search.best_estimator_.get_params())
print('\n',roc_auc_score(y_val,y_preds))

end_time = time.time()
print('\n',time.strftime("%Hh%Mm%Ss", time.gmtime((end_time-start_time))))

# Random Forest optimization

**Optuna SearchCV**

In [ ]:
param_distributions = {
    'criterion': optuna.distributions.CategoricalDistribution(['gini','entropy']),
    'max_features':optuna.distributions.CategoricalDistribution(['auto','sqrt','log2']),
    'max_depth':optuna.distributions.IntUniformDistribution(4,15),
    'n_estimators':optuna.distributions.IntUniformDistribution(150,1500)
}

In [ ]:
start_time = time.time()

rf_opt_search = optuna.integration.OptunaSearchCV(estimator=rf,
                                               param_distributions = param_distributions,
                                               cv=5,
                                               n_jobs=-1,
                                               n_trials = 10,
                                               scoring='roc_auc',
                                               verbose=10)

rf_opt_search.fit(X_train, y_train)
y_preds = rf_opt_search.predict_proba(X_val)[:,1]

print('\n','Best Score',rf_opt_search.best_score_)
print('\n','Best Params',rf_opt_search.best_estimator_.get_params())
print('\n',roc_auc_score(y_val,y_preds))

end_time = time.time()
print('\n',time.strftime("%Hh%Mm%Ss", time.gmtime((end_time-start_time))))

In [ ]:
# Voting
start_time = time.time()

vote_clf = VotingClassifier(estimators=['xgb',xgb_opt_search,
                                        'ada',ada_opt_search,
                                        'rf', rf_opt_search],
                            voting='soft')

vote_clf_fitted = vote_clf.fit(X_train,y_train) # features_final & label here?
preds = vote_clf_fitted.predict_proba(featurez_final)[:,1].reshape(featurez_final.shape[0],1)







end_time = time.time()
print('\n',time.strftime("%Hh%Mm%Ss", time.gmtime((end_time-start_time))))

# SVM optimization

param_distributions = {
    'C':optuna.distributions.UniformDistribution(1,5)
}

from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

#Time = 13 secs
#%%time
voting_predictor(xgb_opt_search,ada_opt_search,rf_opt_search,cols_names=['xgb','ada', 'rf'],
                 file_name='./xgb ada rf opt POS_WEIGHT and fillna feat_mode voting_preds.csv')

start_time = time.time()

svm_opt_search = optuna.integration.OptunaSearchCV(estimator=svm,
                                               param_distributions = param_distributions,
                                               cv=5,
                                               n_jobs=-1,
                                               n_trials = 10,
                                               scoring='roc_auc',
                                               verbose=10)

svm_opt_search.fit(X_train_res, y_train_res)
y_preds = svm_opt_search.predict_proba(X_val)[:,1]

print('\n','Best Score',svm_opt_search.best_score_)
print('\n','Best Params',svm_opt_search.best_estimator_.get_params())
print('\n',roc_auc_score(y_val,y_preds))

end_time = time.time()
print('\n',time.strftime("%Hh%Mm%Ss", time.gmtime((end_time-start_time))))

# Time = 2min
%%time
voting_predictor(xgb_opt_search,svm_opt_search,rf_opt_search,cols_names=['xgb','svm', 'rf'],
                 file_name='./xgb svm rf opt SMOTE corrected and fillna feat_mode voting_preds.csv')

In [ ]:
# Overall time = 

END = time.time()
print('\n',time.strftime("%Hh%Mm%Ss", time.gmtime((END-START))))